# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 2:36PM,239588,19,DEX0006994,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jun 14 2022 2:36PM,239588,19,DEX0006995,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 14 2022 2:36PM,239588,19,DEX0006996,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 14 2022 2:36PM,239588,19,DEX0006997,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 14 2022 2:36PM,239588,19,DEX0006998,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 14 2022 2:35PM,239587,18,CLRSPC-01111280,"ClearSpec, LLC",Released
6,Jun 14 2022 2:28PM,239586,10,Enova-10137,Emerginnova,Released
7,Jun 14 2022 1:53PM,239585,10,SD-061422-951,"SD Head USA, LLC",Released
8,Jun 14 2022 1:46PM,239584,21,617811,"NBTY Global, Inc.",Executing
9,Jun 14 2022 1:35PM,239583,19,ACG-2102490329,ACG North America LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
53,239584,Executing,1
54,239585,Released,1
55,239586,Released,1
56,239587,Released,1
57,239588,Released,5


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239584,NaN,1.0,NaN
239585,NaN,NaN,1.0
239586,NaN,NaN,1.0
239587,NaN,NaN,1.0
239588,NaN,NaN,5.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,35.0,1.0,118.0
239497,0.0,0.0,1.0
239514,0.0,0.0,45.0
239522,0.0,0.0,20.0
239523,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,35,1,118
239497,0,0,1
239514,0,0,45
239522,0,0,20
239523,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,35,1,118
1,239497,0,0,1
2,239514,0,0,45
3,239522,0,0,20
4,239523,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,35,1,118
1,239497,,,1
2,239514,,,45
3,239522,,,20
4,239523,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
5,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC"
6,Jun 14 2022 2:28PM,239586,10,Emerginnova
7,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC"
8,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc."
9,Jun 14 2022 1:35PM,239583,19,ACG North America LLC
11,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC"
12,Jun 14 2022 1:25PM,239579,19,ACG North America LLC
16,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC"
17,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,5
1,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",,,1
2,Jun 14 2022 2:28PM,239586,10,Emerginnova,,,1
3,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC",,,1
4,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",,1,
5,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,,,2
6,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",,,1
7,Jun 14 2022 1:25PM,239579,19,ACG North America LLC,,,4
8,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC",,,1
9,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc.",,24,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5,,
1,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",1,,
2,Jun 14 2022 2:28PM,239586,10,Emerginnova,1,,
3,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC",1,,
4,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",,1,
5,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,2,,
6,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",1,,
7,Jun 14 2022 1:25PM,239579,19,ACG North America LLC,4,,
8,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC",1,,
9,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc.",4,24,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5,,
1,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",1,,
2,Jun 14 2022 2:28PM,239586,10,Emerginnova,1,,
3,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC",1,,
4,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5.0,NaN,NaN
1,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",1.0,NaN,NaN
2,Jun 14 2022 2:28PM,239586,10,Emerginnova,1.0,NaN,NaN
3,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC",1.0,NaN,NaN
4,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 2:36PM,239588,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,5.0,0.0,0.0
1,Jun 14 2022 2:35PM,239587,18,"ClearSpec, LLC",1.0,0.0,0.0
2,Jun 14 2022 2:28PM,239586,10,Emerginnova,1.0,0.0,0.0
3,Jun 14 2022 1:53PM,239585,10,"SD Head USA, LLC",1.0,0.0,0.0
4,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5509652,214.0,33.0,9.0
12,239544,0.0,2.0,0.0
15,958043,158.0,5.0,57.0
18,1197874,5.0,0.0,0.0
19,1916578,16.0,0.0,0.0
20,479086,22.0,36.0,0.0
21,239584,0.0,1.0,0.0
22,239497,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5509652,214.0,33.0,9.0
1,12,239544,0.0,2.0,0.0
2,15,958043,158.0,5.0,57.0
3,18,1197874,5.0,0.0,0.0
4,19,1916578,16.0,0.0,0.0
5,20,479086,22.0,36.0,0.0
6,21,239584,0.0,1.0,0.0
7,22,239497,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,214.0,33.0,9.0
1,12,0.0,2.0,0.0
2,15,158.0,5.0,57.0
3,18,5.0,0.0,0.0
4,19,16.0,0.0,0.0
5,20,22.0,36.0,0.0
6,21,0.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,214.0
1,12,Released,0.0
2,15,Released,158.0
3,18,Released,5.0
4,19,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,9.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0
Executing,33.0,2.0,5.0,0.0,0.0,36.0,1.0,0.0
Released,214.0,0.0,158.0,5.0,16.0,22.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,9.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0
1,Executing,33.0,2.0,5.0,0.0,0.0,36.0,1.0,0.0
2,Released,214.0,0.0,158.0,5.0,16.0,22.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,9.0,0.0,57.0,0.0,0.0,0.0,0.0,0.0
1,Executing,33.0,2.0,5.0,0.0,0.0,36.0,1.0,0.0
2,Released,214.0,0.0,158.0,5.0,16.0,22.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()